In [ ]:
# pip install icecream


In [ ]:
import json,os
# from icecream import ic
import music21 as m21
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical


SAVEFILE_PATH = '/content/drive/MyDrive/MelodyGeneration/'
NOTES_TO_INT_PATH ='/content/drive/MyDrive/MelodyGeneration/mappings/notes_to_int.json'
INT_TO_NOTES_PATH ='/content/drive/MyDrive/MelodyGeneration/mappings/int_to_notes.json'
MODEL_PATH = '/content/drive/MyDrive/MelodyGeneration/Models'
ACCEPTABLE_DURATIONS = [0.25,0.5,0.75,1,2,3,4] #beats
SEQUENCE_LENGTH = 64

In [ ]:
with open(NOTES_TO_INT_PATH,"r") as fp:
  notes_to_int = json.load(fp)

with open(INT_TO_NOTES_PATH,"r") as fp:
  int_to_notes = json.load(fp)

In [ ]:
def generate_medlody(model,seed_sequence,start_symbol,num_steps,max_sequence_length,temperature):
  '''num_steps: No. of the steps in time series representation we want our generator to output,
  max_sequence_lenght : how many steps we want to consider in the seed for the network to be passed as the input(in our case its  64)'''

  def sample_with_temperature(probies, temp):
    prediction = np.log(probies)/ temp
    probies = np.exp(prediction)/np.sum(np.sum(np.exp(prediction)))
    choices = range(len(probies))
    return np.random.choice(choices,p = probies)

  with open(NOTES_TO_INT_PATH,"r") as fp:
    notes_to_int = json.load(fp)
  with open(INT_TO_NOTES_PATH,"r") as fp:
    int_to_notes = json.load(fp)
  unique_notes = len(int_to_notes)
  seed_sequence = seed_sequence.split()
  melody = start_symbol + seed_sequence
  seed_sequence = [notes_to_int[symbol] for symbol in seed_sequence]
  for _ in range(num_steps):
    #taking all the last 64 steps as seed will be growing for num_steps steps
    seed_sequence = seed_sequence[-max_sequence_length:]
    #converting into one hot encoding as our model is trained using these
    onehot_seed = to_categorical(seed_sequence, num_classes = unique_notes)
    #we need to add another dimension because predict() expects 3 dimensions as one dimension tells the batch size
    # in our case the batch size is 1 as we our passing just one sequence for prediction
    onehot_seed = np.reshape(onehot_seed,(1,len(seed_sequence),unique_notes))
    #making prediction
    #now since we have batch size of 1,hence prediction will return list of lenght 1
    probabilities = model.predict(onehot_seed)[0] #[0.2,0.5,0.1,...] size = no. of unique notes
    output_int = sample_with_temperature(probabilities,temperature)
    seed_sequence.append(output_int)
    # now we need to check if the output_int is "/"
    output_symbol = int_to_notes[str(output_int)]
    if output_symbol == "/":
      break
    melody.append(output_symbol)
  return melody

In [ ]:
def MelodyGenerator(model_path,model_type ,sequence_length,seed ):
  model = load_model(os.path.join(model_path,model_type))


  start_symbol = ["/"]* sequence_length

  melody=generate_medlody(model,seed_sequence=seed,
                    start_symbol=start_symbol,
                    num_steps=500,
                    max_sequence_length=sequence_length,
                    temperature=0.7)

  return melody

In [ ]:
model = load_model(os.path.join(MODEL_PATH,"2gru.h5"))

In [ ]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 INPUT_LAYER (InputLayer)    [(None, None, 81)]        0         
                                                                 
 GRU_LAYER_1 (GRU)           (None, None, 256)         260352    
                                                                 
 GRU_LAYER_2 (GRU)           (None, 256)               394752    
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 OUTPUT_LAYER (Dense)        (None, 81)                20817     
                                                                 
Total params: 675921 (2.58 MB)
Trainable params: 675921 (2.58 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.get_metrics_result()['accuracy']

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

In [ ]:
#starting sequence of fur elise
seed_1='76 r 75 76 r 75 r 76 r 71 r _ 74 72 r 69 _ 45 52 r 57 60 r _ '   #FURELISE
seed_2 =

In [ ]:
X = MelodyGenerator(MODEL_PATH,'2gru.h5',SEQUENCE_LENGTH,seed_1)

1/1 [==============================] - 0s 33ms/step


In [ ]:
def save_melody(melody,step_duration = 0.25,format = "midi",filename = 'melody' +'1gru.mid'):
# step_duration: "the amount of duration in a quater length
# that we have at each time step in our reperesentation"

# creating music21 Stream
# we would not be pushing time signature, parts or measures in our stream object
# keeping the things at default set by music21 , ps- ts is 4/4

  stream = m21.stream.Stream()


  #parsing and create note/rest
  #  60 _ _ _ r _ 55
  start_symbol= None #
  step_counter = 1 #its gona keep track of all the steps we have for a event
  #if a step counter is 4 than it would be called a quater lenght note, ie 1 beat duration ie a quarter_note

  for i,symbol in enumerate(melody[64:]):

    #case of notes/rest
    if symbol !="_" or i+1 == len(melody[64:]):
      #but before dealing we want to ensure we are dealing with note/rest after the first one
      if start_symbol is not None:

        quarter_length_duration = step_duration*step_counter
        # ic(start_symbol)
        # ic(symbol)
        #handling rest
        if start_symbol=='r':
          m21_event = m21.note.Rest(length=quarter_length_duration)

        else:
          m21_event = m21.note.Note(int(start_symbol),quarterLength = quarter_length_duration) #number expressing the midi note corresponding to its pitch too

        stream.append(m21_event)
        step_counter = 1


      start_symbol = symbol
    else:
      step_counter +=1

  stream.write(format,os.path.join(SAVEFILE_PATH,filename))

In [ ]:
save_melody(X)

In [ ]:
!yes | add-apt-repository ppa:mscore-ubuntu/mscore3-stable
!apt install musescore3

In [ ]:
song = m21.converter.parse(os.path.join(SAVEFILE_PATH,filename))
song.show()